In [14]:
#https://db.netkeiba.com/race/202005020106/
#馬情報のスクレイピング
import pandas as pd
import numpy as np
import requests
from lxml import html
from bs4 import BeautifulSoup
import tqdm
import concurrent.futures

def get_name_and_url(urls):
    horse = []
    jockey = []
    trainer = []
    for url in urls:
        if '/horse/' in url['href']:
            horse.append({'name' : url.text
                          ,'url' : url['href']})
        elif '/jockey/' in url['href']:
            jockey.append({'name' : url.text
                          ,'url' : url['href']})
        elif '/trainer/' in url['href']:
            trainer.append({'name' : url.text
                          ,'url' : url['href']})
    return horse, jockey, trainer

def create_df(list_dict):
    df = pd.DataFrame(list_dict, columns=['name', 'url'])
    return df

def extract_id(url):
    return url.split('/')[-2]

ID = '202005020102'
URL = f'https://db.netkeiba.com/race/{ID}/'
HTML = requests.get(URL).content
soup = BeautifulSoup(HTML, 'html.parser')
urls = soup.find('table').find_all("a")
horse, jockey, trainer = get_name_and_url(urls)
horse = create_df(horse)
jockey = create_df(jockey)
trainer = create_df(trainer)

In [58]:
# 2010 ~ 2023年
seireki_list = [str(s) for s in list(range(2010, 2024))]
# 01：札幌、02：函館、03：福島、04：新潟、05：東京、06：中山、07：中京、08：京都、09：阪神、10：小倉
spot_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
num_list = ['01', '02', '03', '04', '05', '06', '07']
date_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14']
race_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

id_list = []
for seireki in seireki_list:
    for spot in spot_list:
        for num in num_list:
            for date in date_list:
                for race in race_list:
                    id_list.append(seireki + spot + num + date + race)

In [59]:
def get_trainer_id(ID):
    URL = f'https://db.netkeiba.com/race/{ID}/'
    HTML = requests.get(URL).content
    soup = BeautifulSoup(HTML, 'html.parser')
    urls = soup.find('table').find_all("a")
    horse, jockey, trainer = get_name_and_url(urls)
    trainer = create_df(trainer)
    trainer['trainer_id'] = trainer['url'].map(extract_id)
    trainer['race_id'] = ID
    return trainer

In [76]:
id_list = id_list[:100]

In [77]:
id_list[0] = 0
id_list

[0,
 '201001010102',
 '201001010103',
 '201001010104',
 '201001010105',
 '201001010106',
 '201001010107',
 '201001010108',
 '201001010109',
 '201001010110',
 '201001010111',
 '201001010112',
 '201001010201',
 '201001010202',
 '201001010203',
 '201001010204',
 '201001010205',
 '201001010206',
 '201001010207',
 '201001010208',
 '201001010209',
 '201001010210',
 '201001010211',
 '201001010212',
 '201001010301',
 '201001010302',
 '201001010303',
 '201001010304',
 '201001010305',
 '201001010306',
 '201001010307',
 '201001010308',
 '201001010309',
 '201001010310',
 '201001010311',
 '201001010312',
 '201001010401',
 '201001010402',
 '201001010403',
 '201001010404',
 '201001010405',
 '201001010406',
 '201001010407',
 '201001010408',
 '201001010409',
 '201001010410',
 '201001010411',
 '201001010412',
 '201001010501',
 '201001010502',
 '201001010503',
 '201001010504',
 '201001010505',
 '201001010506',
 '201001010507',
 '201001010508',
 '201001010509',
 '201001010510',
 '201001010511',
 '20100101

In [78]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = [executor.submit(get_trainer_id, id) for id in id_list]
    for result in results:
        print(result)

print("Finished downloading all URLs")
df = pd.DataFrame(columns=['name', 'url', 'trainer_id', 'race_id'])
for result in results:
    df = df.append(result.result())

<Future at 0x1e1744dafc8 state=finished returned DataFrame>
<Future at 0x1e175cf2688 state=finished returned DataFrame>
<Future at 0x1e175c9ba88 state=finished returned DataFrame>
<Future at 0x1e175621f08 state=finished returned DataFrame>
<Future at 0x1e1763fdd48 state=finished returned DataFrame>
<Future at 0x1e176a61048 state=finished returned DataFrame>
<Future at 0x1e1769e9b48 state=finished returned DataFrame>
<Future at 0x1e1760a2c08 state=finished returned DataFrame>
<Future at 0x1e1769e9548 state=finished returned DataFrame>
<Future at 0x1e176a61248 state=finished returned DataFrame>
<Future at 0x1e176367f88 state=finished returned DataFrame>
<Future at 0x1e176609e08 state=finished returned DataFrame>
<Future at 0x1e1767b1d48 state=finished returned DataFrame>
<Future at 0x1e1766eabc8 state=finished returned DataFrame>
<Future at 0x1e175a25e48 state=finished returned DataFrame>
<Future at 0x1e1756eb3c8 state=finished returned DataFrame>
<Future at 0x1e1744cfe08 state=running>


In [79]:
df

,name,url,trainer_id,race_id
0,本田優,/trainer/result/recent/01099/,01099,201001010102
1,高橋成忠,/trainer/result/recent/00216/,00216,201001010102
2,北出成人,/trainer/result/recent/01078/,01078,201001010102
3,伊藤圭三,/trainer/result/recent/01023/,01023,201001010102
4,小島貞博,/trainer/result/recent/01060/,01060,201001010102
...,...,...,...,...
9,佐藤吉勝,/trainer/result/recent/01035/,01035,201001010812
10,池添兼雄,/trainer/result/recent/01021/,01021,201001010812
11,高橋成忠,/trainer/result/recent/00216/,00216,201001010812
12,堀井雅広,/trainer/result/recent/00436/,00436,201001010812


In [80]:
results[0].result()

,name,url,trainer_id,race_id
